# Model Optimizasyon Teknikleri

Bu notebook, yıldız modeli için kullanabileceğiniz model optimizasyon tekniklerini detaylı olarak açıklar. İşte ileri düzey model optimizasyon teknikleri hakkında detaylı bilgi:

## 1. Daha Kapsamlı Hiperparametre Araması

Şu anda `optimize_star_model.py` dosyasında rastgele 10 kombinasyon denenmiş, ancak daha etkin arama yöntemleri kullanabiliriz:

### 1.1 Bayesian Optimizasyon

Bayesian optimizasyon, bir işlevi doğrudan değerlendirmeden değerlerini tahmin etmeye çalışan olasılıksal bir model kurar ve bir kazanım fonksiyonunu eniyilemeye çalışır.

**Avantajları:**
- Verimlilik: Rastgele aramadan daha az deneme ile daha iyi sonuçlar
- Akıllı arama: Önceki denemelerden öğrenir ve umut vadeden bölgeleri keşfeder
- Arama verimliliği: Rastgele arama ve grid search'e göre denemeler arasında daha iyi bir öğrenme eğrisi gösterir

**Nasıl uygulanır:**

In [ ]:
# Bayesian Optimizasyon Örneği
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Keras modelini sarma fonksiyonu
def create_model_wrapper(neurons1=256, neurons2=128, neurons3=64, dropout1=0.4, 
                         dropout2=0.4, dropout3=0.3, learning_rate=0.001):
    def _create_model():
        model = build_star_model(
            n_features, n_classes, 
            neurons1=neurons1, neurons2=neurons2, neurons3=neurons3,
            dropout1=dropout1, dropout2=dropout2, dropout3=dropout3,
            learning_rate=learning_rate
        )
        return model
    return _create_model

# KerasClassifier oluştur
model = KerasClassifier(build_fn=create_model_wrapper())

# Arama alanını tanımla
param_space = {
    'build_fn__neurons1': (128, 512),        # Sürekli aralık
    'build_fn__neurons2': (64, 256),         # Sürekli aralık
    'build_fn__dropout1': (0.2, 0.5),        # Sürekli aralık
    'build_fn__learning_rate': (1e-4, 1e-2), # Logaritmik ölçek
    'batch_size': [32, 64, 128, 256],        # Kategorik
    'epochs': [15, 20, 25]                   # Kategorik
}

# Bayesian aramasını oluştur
bayes_search = BayesSearchCV(
    model, param_space, n_iter=20,          # Sadece 20 kombinasyon dene
    cv=3, scoring='accuracy',
    verbose=1, n_jobs=1, random_state=42
)

# Aramayı gerçekleştir
bayes_search.fit(X_train, y_train)

### 1.2 Grid Search (Izgara Araması)

Grid search, hiperparametre uzayında belirtilen tüm olası kombinasyonları sistematik olarak değerlendirir.

**Avantajları:**
- Kapsamlı: Belirtilen aralıktaki tüm kombinasyonları dener
- Basit: Anlaşılması ve uygulaması kolaydır
- Paralel Çalışma: Paralel işleme çok uygundur

**Dezavantajları:**
- Boyut Laneti: Hiperparametre sayısı arttıkça kombinasyon sayısı üssel olarak artar
- Verimsizlik: Kötü kombinasyonlar için bile zaman harcar

**Nasıl uygulanır:**

In [ ]:
# Grid Search Örneği
from sklearn.model_selection import GridSearchCV

# Kısıtlı parametre alanı (hesaplama süresi için)
param_grid = {
    'build_fn__neurons1': [128, 256, 512],
    'build_fn__neurons2': [64, 128, 256],
    'build_fn__dropout1': [0.3, 0.4, 0.5],
    'build_fn__learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [64, 128],
    'epochs': [15, 20]
}

# Grid search'ü oluştur
grid_search = GridSearchCV(
    model, param_grid, cv=3, scoring='accuracy',
    verbose=1, n_jobs=1
)

# Bazen grid search çok fazla hesaplama gücü gerektirir
# Örneğin burada: 3x3x3x3x2x2 = 324 kombinasyon
# Bu nedenle genellikle hiperparametreleri gruplar halinde optimize ederiz

## 2. Daha Hafif Modeller

### 2.1 DepthwiseSeparableConv 

DepthwiseSeparableConv (Derinliğe göre ayrılabilir evrişim), standart evrişimsel katmanların daha verimli bir versiyonudur. İki adımda çalışır: derinlemesine evrişim ve noktasal evrişim. Bu ayrım, parametre sayısını ve hesaplama yükünü önemli ölçüde azaltır. 

Bu kavram, yoğun (dense) sinir ağları için de uygulanabilir:

**SeparableFC (Ayrılabilir Tam Bağlantılı) Katmanlar:**

In [ ]:
# SeparableFC katmanı uygulaması
import tensorflow as tf
from tensorflow.keras.layers import Layer

class SeparableFC(Layer):
    """Ayırmalabilir tam bağlantılı katman (Dense katmanının hafif versiyonu)"""
    
    def __init__(self, units, rank=8, activation=None, **kwargs):
        super(SeparableFC, self).__init__(**kwargs)
        self.units = units
        self.rank = rank  # Ayırma boyutu
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        
        # İki küçük matris oluşturarak parametre sayısını azalt
        # Standart FC: input_dim * units parametre
        # Ayrılabilir FC: input_dim * rank + rank * units parametre
        
        self.w1 = self.add_weight(
            shape=(input_dim, self.rank),
            initializer='glorot_uniform',
            name='w1'
        )
        self.w2 = self.add_weight(
            shape=(self.rank, self.units),
            initializer='glorot_uniform',
            name='w2'
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            name='bias'
        )
        self.built = True
        
    def call(self, inputs):
        # İki adımda hesapla
        x = tf.matmul(inputs, self.w1)  # (batch, input_dim) x (input_dim, rank)
        x = tf.matmul(x, self.w2)       # (batch, rank) x (rank, units)
        x = tf.nn.bias_add(x, self.bias)
        
        if self.activation is not None:
            x = self.activation(x)
        return x
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [ ]:
# Hafif model yapısında bu katmanı kullanma örneği
def build_lightweight_separable_model(input_dim, n_classes, rank=16):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import BatchNormalization, Dropout, Activation
    
    model = Sequential([
        SeparableFC(256, rank=rank, input_shape=(input_dim,)),
        Activation('relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        SeparableFC(128, rank=rank),
        Activation('relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        SeparableFC(n_classes, rank=rank),
        Activation('softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
    return model

### 2.2 Parametresi Azaltılmış Mimari: Ağaç Yapılı Sinir Ağları

Tüm nöronlar arasında tam bağlantı kurmak yerine, hiyerarşik bağlantılar kuran bir ağaç yapısı kullanılabilir. Bu yaklaşım, uzay jeometrisiyle ilgili özellikle astronomi verilerinde işe yarayabilir.

In [ ]:
# Ağaç yapılı mimarinin Keras ile gerçekleştirilmesi
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, BatchNormalization, Dropout

def build_tree_model(input_dim, n_classes):
    """
    Ağaç yapılı sinir ağı.
    Daha az parametre kullanır ve domain bilgisini modele dahil eder.
    """
    # Girdi
    inputs = Input(shape=(input_dim,))
    
    # İlk seviye: Özellik gruplarını ayır (örn. renk indeksleri, parlaklıklar, vs)
    # Astronomik verilerde bu mantıklı çünkü farklı özellik grupları farklı bilgiler taşır
    
    # Renk indeksi özellikleri (u-g, g-r, r-i, i-z gibi)
    color_feats = Dense(64, activation='relu')(inputs)
    color_feats = BatchNormalization()(color_feats)
    color_feats = Dropout(0.3)(color_feats)
    
    # Parlaklık özellikleri (u, g, r, i, z gibi)
    magnitude_feats = Dense(64, activation='relu')(inputs)
    magnitude_feats = BatchNormalization()(magnitude_feats)
    magnitude_feats = Dropout(0.3)(magnitude_feats)
    
    # Diğer özellikler
    other_feats = Dense(32, activation='relu')(inputs)
    other_feats = BatchNormalization()(other_feats)
    other_feats = Dropout(0.3)(other_feats)
    
    # İkinci seviye: Alt özellikleri birleştir
    combined = concatenate([color_feats, magnitude_feats, other_feats])
    
    # Çıktı katmanı
    outputs = Dense(128, activation='relu')(combined)
    outputs = BatchNormalization()(outputs)
    outputs = Dropout(0.3)(outputs)
    outputs = Dense(n_classes, activation='softmax')(outputs)
    
    # Model oluştur
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
    
    return model

## 3. Erken Durdurma ve Öğrenme Oranı Stratejilerini Optimize Etme

### 3.1 Gelişmiş Erken Durdurma Stratejileri

Standart erken durdurma, doğrulama kaybı iyileşmeyi durduğunda eğitimi sonlandırır. Ancak daha gelişmiş stratejiler kullanabilirsiniz:

In [ ]:
# Gelişmiş erken durdurma ve öğrenme oranı planları
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
import numpy as np

# Trendi izleyen erken durdurma
class TrendingEarlyStopping(Callback):
    """Sadece tek bir plateau için değil, uzun vadeli eğilimi izleyen erken durdurma"""
    
    def __init__(self, monitor='val_loss', patience=5, window_size=10, min_delta=0):
        super(TrendingEarlyStopping, self).__init__()
        self.monitor = monitor
        self.patience = patience
        self.window_size = window_size
        self.min_delta = min_delta
        self.wait = 0
        self.best = float('inf') if 'loss' in monitor else -float('inf')
        self.history = []
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        
        self.history.append(current)
        
        if len(self.history) >= self.window_size:
            # Son window_size değerini kullanarak eğilimi hesapla
            recent = self.history[-self.window_size:]
            # Doğrusal regresyon eğimi
            x = np.arange(len(recent))
            slope = np.polyfit(x, recent, 1)[0]
            
            # 'loss' için negatif eğim iyidir, diğer metrikler için pozitif
            is_improving = slope < -self.min_delta if 'loss' in self.monitor else slope > self.min_delta
            
            if not is_improving:
                self.wait += 1
                if self.wait >= self.patience:
                    self.model.stop_training = True
            else:
                self.wait = 0

### 3.2 Gelişmiş Öğrenme Oranı Planları

ReduceLROnPlateau'dan daha sofistike öğrenme oranı stratejileri kullanabilirsiniz:

In [ ]:
# Döngüsel öğrenme oranı
def cyclic_learning_rate(epoch, min_lr=1e-6, max_lr=1e-3, cycle_length=10):
    """Döngüsel öğrenme oranı planı - ısınma/soğutma döngüleri ile"""
    # Döngü içindeki mevcut konum (0 ile 1 arasında)
    cycle_progress = (epoch % cycle_length) / cycle_length
    
    # Kosinüs dalgası (0 ile 1 arasında, yarım döngü)
    cos_wave = 0.5 * (1 + np.cos(np.pi * (cycle_progress * 2 - 1)))
    
    # Logaritmik ölçekte yumuşak geçiş
    log_min, log_max = np.log10(min_lr), np.log10(max_lr)
    log_lr = log_min + cos_wave * (log_max - log_min)
    
    return 10 ** log_lr

# Isınma dönemi olan adımlı düşüş
def warmup_step_decay(epoch, initial_lr=0.001, min_lr=1e-6, warmup_epochs=3, 
                    drop_epochs=[10, 15, 20], drop_factor=0.2):
    """Isınma dönemi ve önceden belirlenmiş dönemlerde adımlı düşüş"""
    if epoch < warmup_epochs:
        # Isınma dönemi: 0'dan initial_lr'ye doğrusal artış
        return initial_lr * ((epoch + 1) / warmup_epochs)
    
    # Adımlı düşüş
    lr = initial_lr
    for e in drop_epochs:
        if epoch >= e:
            lr *= drop_factor
    
    return max(lr, min_lr)  # Minimum değerden düşük olamaz

In [ ]:
# Bu stratejileri modelde kullanma örneği
def train_with_advanced_strategies(model, X_train, y_train, X_val, y_val, epochs=30):
    # Öğrenme oranı planını ayarla
    lr_scheduler = LearningRateScheduler(
        lambda epoch: cyclic_learning_rate(epoch)
    )
    
    # Gelişmiş erken durdurma
    trend_stopping = TrendingEarlyStopping(
        monitor='val_categorical_accuracy',
        patience=3,
        window_size=8,
        min_delta=0.001
    )
    
    # Model eğitimi
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=128,
        callbacks=[
            lr_scheduler,
            trend_stopping
        ],
        verbose=1
    )
    
    return model, history

## Colab Uygulaması İçin Notlar

Bu optimizasyon stratejilerini Colab'de uygulamak için birkaç ipucu:

1. **Google Colab TPU/GPU Kullanımı**: Colab'de hızlı eğitim için TPU/GPU kullanmayı açın

2. **Eğitim Takibi**: TensorBoard ile eğitim metriklerini görselleştirin

3. **Checkpoint Kaydetme**: Uzun eğitimler için ara modelleri kaydedin

4. **Hyperparameter Tuning**: Colab'de parametre aramayı etkin bir şekilde paralel çalıştırmak için stratejiler

Yukarıdaki teknikleri kullanarak modelinizi hem daha hızlı eğitebilir hem de daha yüksek doğruluk elde edebilirsiniz.